# Part 4 — Object-Oriented ETL Framework

This notebook demonstrates a **reusable OOP design** for ETL pipelines using pandas.

**Design principles:**

| Principle | Description |
|-----------|-------------|
| **Abstract Base Classes** | Define the contract every pipeline must follow |
| **Dependency Injection** | Pass a `DatabaseManager` in; don't create it inside |
| **Plug-in KPIs** | Register KPI classes in a list; add new ones without touching the orchestrator |
| **Template Method** | `BasePipeline.run()` calls extract → transform → build_kpis → load |

**Class hierarchy:**
```
DatabaseManager          — reusable DB I/O
BaseTransformer (ABC)    — override transform()
BaseKPI (ABC)            — override build()
BasePipeline (ABC)       — override extract / get_transformer / get_kpis
```

In [ ]:
import time
import tracemalloc
import yaml
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from datetime import date
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine

# Start performance tracking
tracemalloc.start()
start_time = time.time()

# Configuration
BASE_DIR = Path.cwd()
CREDS_PATH = Path("C:/Users/roger.lloret/Documents/creds/creds_dmbi.yml")
SQL_DIR = BASE_DIR / "sql"

## Reusable: DatabaseManager

A generic database helper. Inject credentials at construction time; reuse the same instance across any number of pipelines.

In [ ]:
class DatabaseManager:
    """
    Generic database helper. Inject credentials at construction time;
    reuse the same instance across any number of pipelines.
    """

    def __init__(self, creds: dict):
        conn_str = (
            f"mysql://{creds['username']}:{creds['password']}"
            f"@{creds['host']}:3306/{creds['database']}"
        )
        self.engine: Engine = create_engine(conn_str)

    def read(self, query: str) -> pd.DataFrame:
        return pd.read_sql(query, self.engine)

    def write(self, df: pd.DataFrame, table_name: str, if_exists: str = "append"):
        with self.engine.begin() as conn:
            df.to_sql(table_name, con=conn, if_exists=if_exists, index=False)

    @staticmethod
    def load_sql(filepath: Path) -> str:
        with open(filepath, "r", encoding="utf-8") as f:
            return f.read()

    @staticmethod
    def load_credentials(path: Path) -> dict:
        if not path.exists():
            raise FileNotFoundError(f"Credentials file not found at {path}")
        with open(path, "r") as f:
            return yaml.safe_load(f)

## Reusable: Abstract Base Classes

These ABCs define the **contract** every pipeline component must follow. Any new pipeline just subclasses and overrides the relevant methods.

In [ ]:
class BaseTransformer(ABC):
    """Override transform() with your domain-specific logic."""

    @abstractmethod
    def transform(self, **dataframes) -> pd.DataFrame:
        """Receive named raw DataFrames, return a single transformed DataFrame."""
        ...


class BaseKPI(ABC):
    """
    Each KPI is its own class. To add a new metric:
      1. Subclass BaseKPI
      2. Implement build()
      3. Register it in the pipeline's get_kpis() list
    """

    @abstractmethod
    def build(self, df: pd.DataFrame) -> pd.DataFrame:
        """Return a DataFrame with columns [kpi_name, kpi_value]."""
        ...


class BasePipeline(ABC):
    """
    Template-method pattern: run() calls extract -> transform -> build_kpis -> load.
    Subclasses only override the parts that change.
    """

    def __init__(self, db: DatabaseManager, table_name: str):
        self.db = db
        self.table_name = table_name

    @abstractmethod
    def extract(self) -> dict[str, pd.DataFrame]:
        """Return a dict of named DataFrames."""
        ...

    @abstractmethod
    def get_transformer(self) -> BaseTransformer:
        """Return the transformer instance for this pipeline."""
        ...

    @abstractmethod
    def get_kpis(self) -> list[BaseKPI]:
        """Return the list of KPI builders to apply."""
        ...

    # --- Shared logic (no need to override) ---

    def build_kpis(self, merged: pd.DataFrame) -> pd.DataFrame:
        """Runs every registered KPI and concatenates results."""
        frames = [kpi.build(merged) for kpi in self.get_kpis()]
        result = pd.concat(frames)
        result["kpi_date"] = date.today()
        return result[["kpi_date", "kpi_name", "kpi_value"]]

    def load(self, df: pd.DataFrame):
        print(f"Loading {len(df)} rows to '{self.table_name}'...")
        self.db.write(df, self.table_name)

    def run(self):
        """Template method — orchestrates the full pipeline."""
        raw = self.extract()
        merged = self.get_transformer().transform(**raw)
        output = self.build_kpis(merged)
        self.load(output)
        print("Pipeline completed successfully.")

## Concrete: InvoiceTransformer

Deduplicates, classifies, casts, and merges invoices + contracts.

In [ ]:
class InvoiceTransformer(BaseTransformer):
    """Deduplicates, classifies, casts, and merges invoices + contracts."""

    @staticmethod
    def classify(df: pd.DataFrame, column: str) -> pd.DataFrame:
        conditions = [
            df[column] < 50,
            (df[column] >= 50) & (df[column] <= 100),
            df[column] > 100,
        ]
        choices = ["Less than 50", "Between 50 and 100", "Greater than 100"]
        df["category"] = np.select(conditions, choices, default="Invalid Input")
        return df

    def transform(self, *, invoices: pd.DataFrame, contracts: pd.DataFrame) -> pd.DataFrame:
        print("Transforming data...")
        contracts = contracts.drop_duplicates()
        invoices = self.classify(invoices, "total_import_euros")
        invoices["contract_id"] = pd.to_numeric(invoices["contract_id"], errors="coerce")
        contracts["contract_id"] = pd.to_numeric(contracts["contract_id"], errors="coerce")
        return invoices.merge(contracts, on="contract_id", how="left")

## Concrete: KPI Definitions

Each KPI is its own class. To add a new metric, just subclass `BaseKPI` and register it.

In [ ]:
class AmountByCategoryKPI(BaseKPI):
    """KPI 1: Total amount by invoice category and client type."""

    def build(self, df: pd.DataFrame) -> pd.DataFrame:
        result = (
            df
            .groupby(["category", "client_type_description"])["total_import_euros"]
            .sum()
            .reset_index()
        )
        result["kpi_name"] = (
            "Total amount in euros of the customers with invoices "
            + result["category"]
            + " euros and "
            + result["client_type_description"]
        )
        return result.rename(columns={"total_import_euros": "kpi_value"})[["kpi_name", "kpi_value"]]


class CountByDocTypeKPI(BaseKPI):
    """KPI 2: Invoice count by document type."""

    def build(self, df: pd.DataFrame) -> pd.DataFrame:
        result = (
            df
            .groupby("document_type_description")["total_import_euros"]
            .count()
            .reset_index()
        )
        result["kpi_name"] = "Number of invoices of invoice type " + result["document_type_description"]
        return result.rename(columns={"total_import_euros": "kpi_value"})[["kpi_name", "kpi_value"]]

## Concrete: InvoiceKPIPipeline

Wires the transformer and KPIs together. To create a different pipeline (e.g., consumption KPIs), subclass `BasePipeline` and override `extract()`, `get_transformer()`, and `get_kpis()`.

In [ ]:
class InvoiceKPIPipeline(BasePipeline):
    """
    Concrete pipeline for invoice KPIs.
    To create a different pipeline (e.g. consumption KPIs):
      1. Subclass BasePipeline
      2. Override extract(), get_transformer(), get_kpis()
      3. Call .run()
    """

    def __init__(self, db: DatabaseManager, sql_dir: Path, table_name: str = "gen_kpi_ft"):
        super().__init__(db, table_name)
        self.sql_dir = sql_dir

    def extract(self) -> dict[str, pd.DataFrame]:
        print("Extracting data from database...")
        invoices_sql = DatabaseManager.load_sql(self.sql_dir / "invoices_main.sql")
        contracts_sql = DatabaseManager.load_sql(self.sql_dir / "contracts_main.sql")
        return {
            "invoices": self.db.read(invoices_sql),
            "contracts": self.db.read(contracts_sql),
        }

    def get_transformer(self) -> BaseTransformer:
        return InvoiceTransformer()

    def get_kpis(self) -> list[BaseKPI]:
        return [
            AmountByCategoryKPI(),
            CountByDocTypeKPI(),
        ]

## Run the Pipeline

Load credentials, create a `DatabaseManager`, and run the pipeline.

In [ ]:
# 1. Load credentials & create a reusable DatabaseManager
creds = DatabaseManager.load_credentials(CREDS_PATH)
db = DatabaseManager(creds["data_warehouse"])

# 2. Create and run the invoice KPI pipeline
pipeline = InvoiceKPIPipeline(db, sql_dir=SQL_DIR)
pipeline.run()

## Performance Metrics

In [ ]:
elapsed = time.time() - start_time
_, peak_memory = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"Pandas (OOP) pipeline executed in {elapsed:.2f} seconds.")
print(f"Peak RAM usage: {peak_memory / (1024 * 1024):.2f} MB")